<a href="https://colab.research.google.com/github/gaurizbburagohain19/GBProject-Game-Recommender/blob/main/game_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install nltk

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [ ]:
from fuzzywuzzy import process

In [ ]:
games = pd.read_csv('/steam.csv')

In [ ]:
games_req = pd.read_csv('/steam_requirements_data.csv')


In [ ]:
games.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99


In [ ]:
games_req.head()


,steam_appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [ ]:
games_req = games_req.rename(columns={'steam_appid': 'appid'})

In [ ]:
games_req.head()

,appid,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [ ]:
games = games.merge(games_req,on='appid')

In [ ]:
games.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,negative_ratings,average_playtime,median_playtime,owners,price,pc_requirements,mac_requirements,linux_requirements,minimum,recommended
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,3339,17612,317,10000000-20000000,7.19,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,633,277,62,5000000-10000000,3.99,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,398,187,34,5000000-10000000,3.99,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,267,258,184,5000000-10000000,3.99,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,288,624,415,5000000-10000000,3.99,{'minimum': '\r\n\t\t\t<p><strong>Minimum:</st...,{'minimum': 'Minimum: OS X Snow Leopard 10.6....,"{'minimum': 'Minimum: Linux Ubuntu 12.04, Dual...","500 mhz processor, 96mb ram, 16mb video card, ...",NaN


In [ ]:
games.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27062 entries, 0 to 27061
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   appid               27062 non-null  int64  
 1   name                27062 non-null  object 
 2   release_date        27062 non-null  object 
 3   english             27062 non-null  int64  
 4   developer           27061 non-null  object 
 5   publisher           27048 non-null  object 
 6   platforms           27062 non-null  object 
 7   required_age        27062 non-null  int64  
 8   categories          27062 non-null  object 
 9   genres              27062 non-null  object 
 10  steamspy_tags       27062 non-null  object 
 11  achievements        27062 non-null  int64  
 12  positive_ratings    27062 non-null  int64  
 13  negative_ratings    27062 non-null  int64  
 14  average_playtime    27062 non-null  int64  
 15  median_playtime     27062 non-null  int64  
 16  owne

In [ ]:
# prompt: delete a column from the dataset

games = games.drop(columns=['english','publisher','required_age','average_playtime','median_playtime','pc_requirements','mac_requirements','linux_requirements','recommended','minimum','owners','platforms','negative_ratings','price'])
games.head()
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27062 entries, 0 to 27061
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   appid             27062 non-null  int64 
 1   name              27062 non-null  object
 2   release_date      27062 non-null  object
 3   developer         27061 non-null  object
 4   categories        27062 non-null  object
 5   genres            27062 non-null  object
 6   steamspy_tags     27062 non-null  object
 7   achievements      27062 non-null  int64 
 8   positive_ratings  27062 non-null  int64 
dtypes: int64(3), object(6)
memory usage: 1.9+ MB


In [ ]:
games.isnull().sum()

,0
appid,0
name,0
release_date,0
developer,0
categories,0
genres,0
steamspy_tags,0
achievements,0
positive_ratings,0


In [ ]:
games.dropna(inplace=True)

In [ ]:
games.duplicated().sum()

np.int64(0)

In [ ]:
games.head(1)

,appid,name,release_date,developer,categories,genres,steamspy_tags,achievements,positive_ratings
0,10,Counter-Strike,2000-11-01,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534


In [ ]:
games['tags']= games['categories'].astype(str) + ','+ games['genres'].astype(str) + ','+ games['steamspy_tags'].astype(str) + ','+ games['release_date'].astype(str) + ','+ games['developer'].astype(str) + ','+ games['positive_ratings'].astype(str) + ','+ games['achievements'].astype(str)

In [ ]:
games['achievements'] = games['achievements'].astype(object)
games['positive_ratings'] = games['positive_ratings'].astype(object)





In [ ]:
games.head()

,appid,name,release_date,developer,categories,genres,steamspy_tags,achievements,positive_ratings,tags
0,10,Counter-Strike,2000-11-01,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,Multi-player;Online Multi-Player;Local Multi-P...
1,20,Team Fortress Classic,1999-04-01,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,Multi-player;Online Multi-Player;Local Multi-P...
2,30,Day of Defeat,2003-05-01,Valve,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,"Multi-player;Valve Anti-Cheat enabled,Action,F..."
3,40,Deathmatch Classic,2001-06-01,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,Multi-player;Online Multi-Player;Local Multi-P...
4,50,Half-Life: Opposing Force,1999-11-01,Gearbox Software,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,Single-player;Multi-player;Valve Anti-Cheat en...


In [ ]:
new_df = games[['appid','name','tags']]

In [ ]:
new_df

,appid,name,tags
0,10,Counter-Strike,Multi-player;Online Multi-Player;Local Multi-P...
1,20,Team Fortress Classic,Multi-player;Online Multi-Player;Local Multi-P...
2,30,Day of Defeat,"Multi-player;Valve Anti-Cheat enabled,Action,F..."
3,40,Deathmatch Classic,Multi-player;Online Multi-Player;Local Multi-P...
4,50,Half-Life: Opposing Force,Single-player;Multi-player;Valve Anti-Cheat en...
...,...,...,...
27057,1065230,Room of Pandora,"Single-player;Steam Achievements,Adventure;Cas..."
27058,1065570,Cyber Gun,"Single-player,Action;Adventure;Indie,Action;In..."
27059,1065650,Super Star Blast,Single-player;Multi-player;Co-op;Shared/Split ...
27060,1066700,New Yankee 7: Deer Hunters,"Single-player;Steam Cloud,Adventure;Casual;Ind..."


In [ ]:
new_df['tags'][0]

'Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled,Action,Action;FPS;Multiplayer,2000-11-01,Valve,124534,0'

In [ ]:
new_df['tags'] = new_df['tags'].astype(str).str.replace(';', '|', regex=False)

/tmp/ipython-input-32-497508985.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].astype(str).str.replace(';', '|', regex=False)


In [ ]:
new_df['tags'][0]

'Multi-player|Online Multi-Player|Local Multi-Player|Valve Anti-Cheat enabled,Action,Action|FPS|Multiplayer,2000-11-01,Valve,124534,0'

In [ ]:
new_df['tags'] = new_df['tags'].astype(str).str.replace(',', ' ; ', regex=False)

/tmp/ipython-input-34-3603644884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].astype(str).str.replace(',', ' ; ', regex=False)


In [ ]:
new_df['tags'][0]

'Multi-player|Online Multi-Player|Local Multi-Player|Valve Anti-Cheat enabled ; Action ; Action|FPS|Multiplayer ; 2000-11-01 ; Valve ; 124534 ; 0'

In [ ]:
new_df['tags'] = new_df['tags'].astype(str).str.replace('|', ' , ', regex=False)

/tmp/ipython-input-36-1999394726.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].astype(str).str.replace('|', ' , ', regex=False)


In [ ]:
new_df['tags'] = new_df['tags'].astype(str).str.replace('-', '', regex=False)

/tmp/ipython-input-37-1015313825.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].astype(str).str.replace('-', '', regex=False)


In [ ]:
new_df['tags'][0]


'Multiplayer , Online MultiPlayer , Local MultiPlayer , Valve AntiCheat enabled ; Action ; Action , FPS , Multiplayer ; 20001101 ; Valve ; 124534 ; 0'

In [ ]:
ss = SnowballStemmer("english")

In [ ]:
def stem(text):
  y = []
  for i in text.split():
     if i.isalpha():                     # only stem alphabetic words
            y.append(ss.stem(i))
     else:
            y.append(i)                     # keep numbers or punctuation as-is
  return " ".join(y)

In [ ]:
new_df['tags'].apply(stem)

,tags
0,"multiplay , onlin multiplay , local multiplay ..."
1,"multiplay , onlin multiplay , local multiplay ..."
2,"multiplay , valv anticheat enabl ; action ; fp..."
3,"multiplay , onlin multiplay , local multiplay ..."
4,"singleplay , multiplay , valv anticheat enabl ..."
...,...
27057,"singleplay , steam achiev ; adventur , casual ..."
27058,"singleplay ; action , adventur , indi ; action..."
27059,"singleplay , multiplay , coop , Shared/Split s..."
27060,"singleplay , steam cloud ; adventur , casual ,..."


In [ ]:
stem('Multiplayer , Online MultiPlayer , Local MultiPlayer , Valve AntiCheat enabled ; Action ; Action , FPS , Multiplayer ; 20001101 ; Valve ; 124534 ; 3339 ; windows , mac , linux ; 1000000020000000 ; 7.19')

'multiplay , onlin multiplay , local multiplay , valv anticheat enabl ; action ; action , fps , multiplay ; 20001101 ; valv ; 124534 ; 3339 ; window , mac , linux ; 1000000020000000 ; 7.19'

In [ ]:
new_df['tags'] = new_df['tags'].fillna('').astype(str)

/tmp/ipython-input-44-4237316557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['tags'] = new_df['tags'].fillna('').astype(str)


In [ ]:
new_df = new_df[new_df['tags'].str.strip() != '']

In [ ]:
cv = CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
vectors= cv.fit_transform(new_df['tags']).toarray()

In [ ]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
vectors[0]

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
cv.get_feature_names_out()

array(['07th', '10', '100', ..., '自由愉悦小黄猫', '致意', '迷茫的菜鸟'], dtype=object)

In [ ]:
model = NearestNeighbors(n_neighbors=11, algorithm ='brute',metric='cosine')
model.fit(vectors)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=11)

In [ ]:
def recommend(game_name):
    game_name = game_name.lower()
    match = new_df[new_df['name'].str.lower() == game_name]

    if match.empty:
        print(f"Game '{game_name}' not found.")
        return

    index = match.index[0]
    distances, indices = model.kneighbors(vectors[index].reshape(1, -1))

    print(f"\nRecommendations similar to: {new_df.iloc[index]['name']}\n")
    for i, dist in zip(indices[0][1:], distances[0][1:]):
        similarity = 1 - dist
        print(f"{new_df.iloc[i]['name']} (Similarity: {similarity:.2f})")


the recommend function below is better as it uses FuzzyWuzzy

In [ ]:
def recommend(game_name):
    # Fuzzy to match the name of game
    game_names = list(new_df['name'])
    match = process.extractOne(game_name, game_names)

    if match[1] < 60:
        print(f"No good match found for '{game_name}'.")
        return

    index = new_df[new_df['name'] == match[0]].index[0]

    distances, indices = model.kneighbors(vectors[index].reshape(1, -1))

    print(f"\nRecommendations similar to: {match[0]}\n")
    for i in indices[0]:
        print(new_df.iloc[i]['name'])



In [ ]:
recommend('Counter-Strike')


Recommendations similar to: Counter-Strike

Team Fortress Classic
Deathmatch Classic
Counter-Strike
Ricochet
Half-Life Deathmatch: Source
Counter-Strike: Condition Zero
Half-Life 2: Deathmatch
Half-Life
Day of Defeat
Counter-Strike: Source
Battlefield: Bad Company™ 2


In [ ]:
recommend('Red Dead Redemption 2')


Recommendations similar to: Redemption

Redemption
HALF DEAD 2
Bloodlines of Prima
Batch 17
DiveReal
Magicae Mundi
Fear the Night - 恐惧之夜
Hide and Seek
INTERSHELTER
Scribbled Arena
Elemental World Part 1:Rise Of The Guardians


In [ ]:
recommend('Grand Theft Auto V')


Recommendations similar to: Grand Theft Auto V

Grand Theft Auto V
Just Cause 2: Multiplayer Mod
Grand Theft Auto IV
Grand Theft Auto: Episodes from Liberty City
Assassin’s Creed® Brotherhood
Watch_Dogs™
Sniper Ghost Warrior 3
Grand Theft Auto: San Andreas
Bully: Scholarship Edition
Man in a Maze: Deathmatch
NARUTO SHIPPUDEN: Ultimate Ninja STORM 3 Full Burst HD
